In [ ]:
%matplotlib inline
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from typing import Optional, Tuple, Dict
import MDAnalysis as mda
from MDAnalysis.analysis.base import AnalysisFromFunction
import numpy as np
import pandas as pd
from MDAnalysis import transformations as mda_trans
from MDAnalysis.analysis import distances as mda_dist
import os
from polyphys.manage.typer import ParserT
from polyphys.manage.parser import SumRuleCyl, TransFociCyl, TransFociCub, Dump
from polyphys.manage.organizer import invalid_keyword, sort_filenames
from polyphys.analyze import clusters
from polyphys.analyze import correlations
from polyphys.probe import prober
from polyphys.probe.prober import stamps_report
import warnings

from MDAnalysis.analysis import polymer
from MDAnalysis.analysis import distances
from polyphys.manage.parser import HnsCub
from polyphys.manage import organizer

In [ ]:
import MDAnalysis as mda
import MDAnalysis.analysis.msd as msd

In [ ]:
attr_labels = {
    'phi_c_bulk_round': "$\phi_c$",
    'time': "$\hat{{t}}$",
    'time_norm': "$\hat{{t}}/\hat{{t}}^{{max}}$",
    'lags': "$\hat{t}_{lags}$",
    'dmon_large': "${{a_M}}/{{a_m}}$",
    'dcrowd': "${{a_c}}/{{a_m}}$",
    'dcyl': "${{D}}/{{a_m}}$",
    'nmon_small': "$N_m$",
    'nmon_large': "$N_M$",
    'phi_c_bulk_norm': "${a\phi_c}/{a_c}$",
    'genomic_distance': '$\Delta n$',
    'bin_center': '$r$',
    'bin_center-norm-dmon_large': '${{r}}/{{a_M}}$',
    'bin_center-norm': '${{r}}/{{r_{max}}}$',
    'bin_center-r': '$r$',
    'bin_center-norm-r': '${{2r}}/{{D}}$',
    'bin_center-recentered-norm-r': '${{(2r-a^{shift})}}/{{D}}$',
    'bin_center-dcrowd-r': '${{2r}}/{{a_c}}$',
    'bin_center-dcrowd-recentered-r': '${{(2r-a^{shift})}}/{{a_c}}$',
    'bin_center-z': '$z$',
    'bin_center-norm-z': '${{|z|}}/{{z_{max}}}$',
    'bin_center-dcrowd-z': '${{|z|}}/{{z_{max}}}$',
    'bin_center-dcrowd-recentered-z': '${{|z|}}/{{z_{max}}}$',
    'bin_center-theta': '$\theta$',
    'bin_center-dcrowd-theta': '${{\theta}}/{{\pi}}$',
    'bin_center-norm-theta': '${{\theta}}/{{\pi}}$',
    'nmon': '$N$',
    'nhns': '$N_{{hns}}$',
    'phi_c_rescaled': '$\\frac{{a\phi_c}}{{a_c}}$',
    'confinement_rate': '$\kappa=\\frac{{a_c}}{{D-a_c}}$',
    'confinement_rate_r': '$\kappa=\\frac{{D-a_c}}{{a_c}}$',
    'dep_energy_max': '$\mathcal{F}_{dep}=\phi_c^{(bulk)}[1+{{3a_m}}{{2a_c}}]$',
    'int_energy_max': '$\mathcal{F}_{int}=\\frac{{Na\phi_c^{(bulk)}}}{{a_c}}[3a_ma_c + \\frac{{3}}{{2}}]$',
    'species': 'Type'
}

### Probe explorations

In [ ]:
# analyzing bug files.
group = 'nucleoid'
lineage = 'whole'
save_to = './'
#path = "/Users/amirhsi/OneDrive - University of Waterloo/PhD Research/Jupyter/N200epshm29kbmm10nh0ac2nc0l50dt0.005ndump2000adump5000ens1.ring"
#path = "/Users/amirhsi_mini/OneDrive - University of Waterloo/PhD Research/Jupyter/N200epshm29kbmm20nh48ac2nc0l60dt0.005ndump2000adump5000ens1.ring"
#path = "/Users/amirhsi_mini/research_data/N200epshm29kbmm10nh0ac2nc0l50dt0.005ndump2000adump5000ens1.ring"
#path = "../../Datasets/N200epshm29kbmm2nh48ac1nc95493l25dt0.005ndump2000adump5000ens1.ring"
#path = "/Users/amirhsi_mini/research_data/N200epshm29kbmm2nh0ac2nc0l25dt0.005ndump2000adump5000ens1.ring"
path = "../../N200epshm29kbmm2nh0ac2nc0l25dt0.005ndump2000adump5000ens1.ring"
nuc_pairs = glob(path + '/N*' + group + '*')
nuc_pairs = sort_filenames(
    nuc_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)
nuc_pairs

In [ ]:
%%time
# 50 mins
for nuc_pair in nuc_pairs[:1]:
    topology = nuc_pair[0]
    trajectory = nuc_pair[1]
    lineage = 'whole'
    save_to = './'
    continuous = False
    if (lineage == 'segment') & (continuous is False):
        warnings.warn(
            "lineage is "
            f"'{lineage}' "
            "and 'continuous' is "
            f"'{continuous}. "
            "Please ensure the "
            f"'{trajectory}' is NOT part of a sequence of trajectories.",
            UserWarning
        )
    print("Setting the name of analyze file...")
    sim_info = HnsCub(
        trajectory,
        lineage,
        'cubic',
        'nucleoid',
        'ring'
    )
    sim_name = sim_info.lineage_name + "-" + sim_info.group
    print("\n" + sim_name + " is analyzing...\n")
    # LJ time difference between two consecutive frames:
    time_unit = sim_info.dmon * np.sqrt(
        sim_info.mmon * sim_info.eps_others)  # LJ time unit
    # Sampling via LAMMPS dump every 'ndump', so trajectory dt is:
    sim_real_dt = sim_info.ndump * sim_info.dt * time_unit
    cell = mda.Universe(
        topology, trajectory, topology_format='DATA',
        format='LAMMPSDUMP', lammps_coordinate_convention='unscaled',
        unwrap_images=True,
        atom_style="id resid type x y z", dt=sim_real_dt,
        )
    if continuous:
        sliced_trj = cell.trajectory[0: -1]
        n_frames = cell.trajectory.n_frames - 1
    else:
        sliced_trj = cell.trajectory[::10000]
        n_frames = cell.trajectory.n_frames
        n_frames = len(sliced_trj)
    # selecting atom groups:
    bug = cell.select_atoms('resid 1')  # the bug
    hns_patch = cell.select_atoms('type 2')  # the hns patches
    # transformations:
    workflow = [
        mda_trans.unwrap(cell.atoms),
        mda_trans.center_in_box(bug),
        mda_trans.wrap(cell.atoms)
    ]
    cell.trajectory.add_transformations(*workflow)
    # defining collectors
    # bug:
    gyr_t = []
    principal_axes_t = np.empty([0, 3, 3])
    asphericity_t = []
    shape_parameter_t = []
    # - bond info
    n_bonds = len(bug.bonds.indices)
    bond_lengths = np.zeros((n_bonds,1), dtype=np.float64)
    cosine_corrs = np.zeros(n_bonds, dtype=np.float64)
    # mon and hns-patch are attracted if their distance <= the below distance:
    m_hpatch_attr_cutoff = 0.5 * (sim_info.dmon + sim_info.dhns_patch)
    m_hpatch_shape = (sim_info.nmon, 2 * sim_info.nhns)
    m_m_shape = (sim_info.nmon, sim_info.nmon)
    # distance matrices
    dist_m_hpatch = np.zeros(m_hpatch_shape, dtype=np.float64)
    dist_m_m = np.zeros(m_m_shape, dtype=np.float64)
    # contact matrices
    dir_contacts_m_hpatch = np.zeros(m_hpatch_shape, dtype=np.int64)
    dir_contacts_m_m = np.zeros(m_m_shape, dtype=np.int64)
    for _ in sliced_trj:
        # bug:
        # -various measures of chain size
        gyr_t.append(bug.radius_of_gyration())
        # -shape parameters:
        asphericity_t.append(bug.asphericity())
        principal_axes_t = np.append(
            principal_axes_t,
            np.array([bug.principal_axes()]),
            axis=0
        )
        shape_parameter_t.append(bug.shape_parameter())
        # -bond info
        bond_dummy, cosine_dummy = correlations.bond_info(
            bug,
            sim_info.topology
            )
        bond_lengths += bond_dummy
        cosine_corrs += cosine_dummy
        # bug - hns patch:
        # - distance matrices
        dummy = mda_dist.distance_array(bug, hns_patch, box=cell.dimensions)
        print(dummy.shape)
        dist_m_hpatch += dummy
        dummy_m_m = np.matmul(dummy, dummy.T)
        dist_m_m += dummy_m_m
        # - contact matrices
        dummy = np.asarray(dummy <= m_hpatch_attr_cutoff, dtype=int)
        dir_contacts_m_hpatch += dummy
        dummy_m_m = np.matmul(dummy, dummy.T)
        dir_contacts_m_m += dummy_m_m
    # Saving collectors to memory
    # bug
    np.save(save_to + sim_name + '-gyrTMon.npy', np.array(gyr_t))
    outfile = save_to + sim_name + "-stamps.csv"
    stamps_report(outfile, sim_info, n_frames)
    np.save(save_to + sim_name + '-asphericityTMon.npy',
            np.array(asphericity_t)
            )
    np.save(save_to + sim_name + '-principalTMon.npy', principal_axes_t)
    np.save(save_to + sim_name + '-shapeTMon.npy', shape_parameter_t)
    bond_lengths = bond_lengths / n_frames
    bonds_per_lag = np.arange(n_bonds, 0, -1)
    cosine_corrs = cosine_corrs / (n_frames * bonds_per_lag)
    np.save(save_to + sim_name + '-bondLengthVecMon.npy', bond_lengths)
    np.save(save_to + sim_name + '-bondCosineCorrVecMon.npy', cosine_corrs)
    # bug hns-patch:
    dist_m_hpatch = dist_m_hpatch / n_frames
    dist_m_m = dist_m_m / n_frames
    dir_contacts_m_hpatch = dir_contacts_m_hpatch / n_frames
    dir_contacts_m_m = dir_contacts_m_m / n_frames
    np.save(
        save_to + sim_name + "-distMatTMonPatch.npy", dist_m_hpatch
    )
    np.save(
        save_to + sim_name + "-distMatTMonMon.npy", dist_m_m
    )
    np.save(
        save_to + sim_name + "-directContactsMatTMonPatch.npy",
        dir_contacts_m_hpatch
    )
    np.save(
        save_to + sim_name + "-directContactsMatTMonMon.npy",
        dir_contacts_m_m
    )
    # Simulation stamps:
    print('done.')

### MSD of monomers: g_1(t)

In [ ]:
MSD = msd.EinsteinMSD(cell, select='resid 1', msd_type='xyz', fft=True)
MSD.run()

In [ ]:
mda.__version__

In [ ]:
core_msd = MSD.results.timeseries
#core_msd

In [ ]:
nframes = MSD.n_frames
timestep = 1 # this needs to be the actual time between frames
lag_times = np.arange(nframes)*timestep # make the lag-time axis
fig = plt.figure()
ax = plt.axes()
# plot the actual MSD
ax.plot(lag_times, core_msd)#, lc="black", ls="-", label=r'3D random walk')
#exact = lag_times*6
# plot the exact result
#ax.plot(lag_times, exact, lc="black", ls="--", label=r'$y=2 D\tau$')

plt.show()

### g_2(t): mean-square-displacement in center of gravity of chain in the center of mass reference

In [ ]:
#time = []
rgyr = []
for ts in cell.trajectory:
    #if ts.time == 0:
    #if ts ==
    #time.append(cell.trajectory.time)
    rgyr.append(bug.radius_of_gyration())


In [ ]:
np.asarray(rgyr).mean()

In [ ]:
def gyr_squared(group, gyr_0, wrap=False):
    """Radius of gyration.
    Parameters
    ----------
    gyr_0
    group
    wrap : bool, optional
        If ``True``, move all atoms within the primary unit cell before
        calculation. [``False``]
    .. versionchanged:: 0.8
       Added `pbc` keyword
    .. versionchanged:: 2.1.0
       Renamed `pbc` kwarg to `wrap`. `pbc` is still accepted but
       is deprecated and will be removed in version 3.0.
    """
    atomgroup = group.atoms
    masses = atomgroup.masses
    com = atomgroup.center_of_mass(wrap=wrap)
    if wrap:
        recentered_pos = atomgroup.pack_into_box(inplace=False) - com
    else:
        recentered_pos = atomgroup.positions - com - gyr_0
    rog_sq = np.sum(masses * np.sum(recentered_pos**2,
                                    axis=1)) / atomgroup.total_mass()
    return rog_sq

In [ ]:
#ljcut_coeff = 1.122462
#hns_core_mon_cutoff = 0.5*(sim_info.dmon + sim_info.dhns) #* 5# * ljcut_coeff
hns_pole_mon_cutoff = 0.5*(sim_info.dmon + 0.178) #* ljcut_coeff
#mon_mon_cutoff = 0.5*(sim_info.dmon*ljcut_coeff)
#dist_array_core_contact = np.zeros((sim_info.nmon, sim_info.nhns), dtype=np.int64)
#dist_array_pole_contact = np.zeros((sim_info.nmon, 2*sim_info.nhns), dtype=np.int64)
#dist_array_mon_contact = np.zeros((sim_info.nmon, sim_info.nmon), dtype=np.int64)


In [ ]:
contact_m_m.sum()

In [ ]:
np.trace(contact_m_m)

In [ ]:
%%time
for ts in sliced_trj:
    # bug:
    #dist_m_hcore = distances.distance_array(bug, hns_core, box=cell.dimensions)
    #contact_m_hcore = np.asarray(dist <= hns_core_mon_cutoff, dtype=int)
    #contact_m_m = np.matmul(contact_m_hcore, contact_m_hcore.T)
    dist_m_pcore = distances.distance_array(bug, hns_pole, box=cell.dimensions)
    contact_m_pcore = np.asarray(dist_m_pcore <= hns_pole_mon_cutoff, dtype=int)
    contact_m_m = np.matmul(contact_m_pcore, contact_m_pcore.T)
    contact_m_m_acc += contact_m_m
    #dummy = distances.distance_array(bug, hns_core, box=cell.dimensions)
    #dummy = np.asarray(dummy <= hns_core_mon_cutoff, dtype=int)
    #dist_array_core_contact += dummy
    #dummy = distances.distance_array(bug, hns_pole, box=cell.dimensions)
    #dummy = np.asarray(dummy <= hns_pole_mon_cutoff, dtype=int)
    #dist_array_pole_contact += dummy
    #dummy = distances.distance_array(bug, bug, box=cell.dimensions)
    #dummy = np.asarray(dummy <= mon_mon_cutoff, dtype=int)
    #dist_array_mon_contact += dummy

#dist_array_core_contact_mean = dist_array_core_contact / n_frames
#dist_array_pole_contact_mean = dist_array_pole_contact / n_frames
#dist_array_mon_contact_mean = dist_array_mon_contact / n_frames
#np.save(
#    save_to + sim_name + "distCoreMonMean.npy",
#    dist_array_core_contact_mean
#)
#np.save(
#    save_to + sim_name + "-distPoleMonMean-New.npy",
#    dist_array_pole_contact_mean
#)
contact_m_m_acc = contact_m_m_acc / n_frames
np.save(
    save_to + sim_name + "-ContactMapMonMonMean-New.npy",
    contact_m_m_acc
)
#np.save(
#    save_to + sim_name + "distMonMonMean.npy",
#    dist_array_pole_contact_mean
#)

In [ ]:
sns.heatmap(contact_m_m_acc, cmap='rocket_r',# vmin=0, vmax=0.03
                     )

In [ ]:
glob("./N200*"+phys_attrs[attr]+".npy")[-2]

In [ ]:
np.load(glob("./N200*"+phys_attrs[attr]+".npy")[-2]).shape

## Visualizing contacts:

In [ ]:
phys_attrs = {
    #"Core": "distCoreMonMean-cutoff*",
    #"Pole": "distPoleMonMean-cutoff*"
    "Core-Mon": "ContactMonMon-Core-cutoff*",
    "Pole-Mon": "ContactMonMon-Pole-cutoff*"

}
attr = "Core-Mon"
dists = glob("./N200*"+phys_attrs[attr]+".npy")
dists = sorted(dists)
dists = dists
dist_lists = []
for dist in dists:
    dist_info = HnsCub(
        dist,
        'whole',
        'cubic',
        'nucleoid',
        'ring'
    )

    dist_lists.append([dist_info.nhns, dist_info.dcrowd, round(dist_info.phi_c_bulk,2), dist_info.nmon, dist_info.filepath])
dist_lists.sort(key=lambda dist: (dist[0], dist[2]))

In [ ]:
rc_params = {
    'mathtext.default': 'regular',
    #'text.usetex': True
}
sns.set_theme(
 #   context='paper',
    font='Times New Roman',
    font_scale=1.5,
    rc=rc_params
)
fig, axes = plt.subplots(nrows=5, ncols=2, sharey=True, figsize=(15,20))
#cbar_ax = fig.add_axes([.91, .2, .03, .6])
nmon = 200
for idx, (dist, ax) in enumerate(zip(dist_lists, axes.flatten(order='F'))):
    pole_data = np.load(dist[-1])
    if attr in ["Core-Mon", "Pole-Mon"]:
        np.fill_diagonal(pole_data, 0)
    ax_hmap = sns.heatmap(
        pole_data, ax=ax, cmap='rocket_r', #vmin=0, vmax=0.02, # Pole
        #pole_data, ax=ax, cmap='rocket_r', vmin=0, vmax=0.045,  # Core
        #square=True
        #cbar_kws={'label': 'Normalized contacts'},
        #cbar= idx > 4,
        #cbar_ax= None if idx==0 else cbar_ax,
    )

    #ax.set_xticks(np.arange(0.5,2*dist[0]+0.5,5))
    #ax.set_xticklabels(np.arange(1,2*dist[0],5))
    #ax.set_xticks(np.arange(0.5,dist[0]+0.5,5))
    #ax.set_xticklabels(np.arange(1,dist[0],5))
    ax.set_xticks(np.arange(0.5,dist[3]+0.5,30))
    ax.set_xticklabels(np.arange(1,dist[3],30))
    ax.set_yticks(np.arange(0.5,dist[3],30))
    ax.set_yticklabels(np.arange(1,dist[3],30))
    if idx < 5:
        ax.set_ylabel('Monomer (index)')
    if idx % 5 == 4:
        ax.set_xlabel(f'Hns {attr} (index)')
    ax.set_title(fr'$n_{{hns}}={dist[0]}, \phi_c={dist[2]}, a_c={dist[1]}$')
fig.suptitle(
    r"Normalized number of contacts between monomers and hns " \
        + attr.lower() +  "s in $10^5$ counts"
)
fig.tight_layout(h_pad=1, w_pad=0)#rect=[0, 0, .9, 1])
fig.savefig("ContactMap-"+attr+"Mon-cutoff1.0.pdf")

# Visualizing persistence length

In [ ]:
import scipy
def fit_exponential_decay(x, y):
    r"""Fit a function to an exponential decay

    .. math::  y = \exp\left(- \frac{x}{a}\right)

    Parameters
    ----------
    x, y : array_like
      The two arrays of data

    Returns
    -------
    a : float
      The coefficient *a* for this decay

    Notes
    -----
    This function assumes that data starts at 1.0 and decays to 0.0

    """
    def exp_func(x, a):
        return np.exp(-x/a)
    a = scipy.optimize.curve_fit(exp_func, x, y)[0][0]
    return a

In [ ]:
# LOad data
allInOne_db = '/Users/amirhsi_mini/research_data/analysis/'
group = 'nucleoid'
geometry = 'cubic'
topology = 'ring'
project = 'HnsCub'
bond_corr = '-'.join(['allInOne', project, group, 'chainBondCosCorr.csv'])
bond_corr = allInOne_db + bond_corr
bond_corr = pd.read_csv(bond_corr)
bond_corr.drop(
    columns=['bondCosineCorrVecMon-var','bondCosineCorrVecMon-sem'],
    inplace =True
)

In [ ]:
bond_corr_ens[bond_corr_ens.bin_center>100]

In [ ]:
# a sorted list of unique spaces in the dataset
ensemble_longs = bond_corr['ensemble_long'].unique()
project_titles = {}
bond_corr['exp_fit'] = 0
for ensemble_long in ensemble_longs:
    s_info = HnsCub(
        ensemble_long,
        'ensemble_long',
        geometry,
        group,
        topology,
        ispath=False
    )
    bond_corr_ens = bond_corr[bond_corr['ensemble_long']==ensemble_long]
    bond_corr_ens_idx = bond_corr_ens.index
    per_length = fit_exponential_decay(
        bond_corr_ens['bin_center'],
        bond_corr_ens['bondCosineCorrVecMon-mean']
    )
    bond_corr.loc[bond_corr_ens_idx,'exp_fit'] = \
        bond_corr_ens.loc[:, 'bin_center'].apply(lambda x: np.exp(-x/per_length))


In [ ]:
y_prop = 'bondCosineCorrVecMon-mean'
col_attr = 'nhns'
col_order = sorted(bond_corr[col_attr].unique())
hue_attr = 'phi_c_bulk_round'
hue_order = sorted(bond_corr[hue_attr].unique())
style_attr = 'dcrowd'
style_order = sorted(bond_corr[style_attr].unique())
color_palette = 'flare'
#color_palette = 'tab10'
#color_palette = ['orange', 'darkgoldenrod', 'forestgreen']
save_to = './'
ext = 'pdf'
plot_context = 'paper'
font_scale = 2
height = 4
aspect = 1.618
ylabel_pad = 50
rc_params= {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'axes.grid.axis': 'both',
    #'axes.grid.which': 'both'
}
facet_kws = {
    'sharey': False,
    'sharex': False,
    'legend_out': True,
}
fig_title_kws = {'fontsize': 34, 'x': 0.5, 'y': 0.93}
loc='lower left'
font_family = 'sans-serif'
move_legend_kws = {
    'ncol': 1,
    'bbox_to_anchor': (1., 0.95),
    'frameon': True,
    'facecolor': 'aliceblue',
    #'borderpad': 0.1,
    'markerscale': 1.5
}
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)
bond_grid = sns.relplot(
    data=bond_corr,
    x='bin_center',
    y=y_prop,
    row=col_attr,
    row_order=col_order,
    hue=hue_attr,
    hue_order=hue_order,
    style=style_attr,
    style_order=style_order,
    ci=None,
    kind='line',
    markers=False,
    legend='full',
    height=height,
    aspect=aspect,
    palette=color_palette,
    facet_kws=facet_kws,
)
bond_grid.set_xlabels('Bond index lag, j')
#bond_grid.set_ylabels(r'$\langle\cos(\theta_{i,i+j})\rangle$')
bond_grid.set_ylabels(r'$\langle\hat{l}_i.\hat{l}_{i+j}\rangle$')
bond_grid.set_titles(r'$N=200, n_{{hns}}={row_name}$')
legend_labels = [attr_labels[hue_attr]] + list(hue_order)  + \
        [attr_labels[style_attr]] + list(style_order)
for idx, new_label in enumerate(legend_labels):
    bond_grid._legend.texts[idx].set_text(new_label)
#bond_grid.tight_layout(w_pad=1)
sns.move_legend(bond_grid, loc='upper left',  **move_legend_kws)
output = "-".join(["equilPlot", project, col_attr, hue_attr, 'bondCosineCorr'])
bond_grid.fig.tight_layout(pad=1)
bond_grid.savefig(save_to + output + '.' + ext, bbox_inches='tight', dpi=300)

In [ ]:
corrs = glob("./N200*-bondCosineCorrsVec.npy")
corrs = sorted(corrs)
corr_lists = []
for corr in corrs:
    corr_info = HnsCub(
        corr,
        'whole',
        'cubic',
        'nucleoid',
        'ring'
    )
    corr_lists.append([corr_info.nhns, corr_info.dcrowd, round(corr_info.phi_c_bulk,2), corr_info.nmon, corr_info.filepath])
corr_lists.sort(key=lambda dist: (dist[0], dist[2]))

In [ ]:
fit_exponential_decay(bond_idx, cosine_corrs_acc/n_bonds/n_frames)

In [ ]:
corr_data = np.load(corr[-1])
bonds_per_lag = np.arange(n_bonds,0, -1)
corr_data = corr_data/bonds_per_lag
bond_idx = np.arange(corr[3])
per_length, b, c = fit_exponential_decay(bond_idx[:n_bonds//2], corr_data[:n_bonds//2])
bond_corr = np.exp(-c*bond_idx**b/per_length)
print(per_length, b, c)

In [ ]:
rc_params = {
    'mathtext.default': 'regular',
    #'text.usetex': True
}
sns.set_theme(
    context='paper',
    font='Times New Roman',
    font_scale=1.5,
    rc=rc_params
)
fig, axes = plt.subplots(nrows=6, ncols=2, sharey=True, figsize=(15,20))
#cbar_ax = fig.add_axes([.91, .2, .03, .6])
nmon = 200
for idx, (corr, ax) in enumerate(zip(corr_lists, axes.flatten(order='F'))):
    corr_data = np.load(corr[-1])
    corr_data = np.append(corr_data,corr_data[0])
    bond_idx = np.arange(1,corr[3]+2,1)
    ax_hmap = ax.scatter(
        bond_idx, corr_data,
        #square=True
        #cbar_kws={'label': 'Normalized contacts'},
        #cbar= idx > 4,
        #cbar_ax= None if idx==0 else cbar_ax,
    )

    #ax.set_xticks(np.arange(0.5,2*dist[0]+0.5,5))
    #ax.set_xticklabels(np.arange(1,2*dist[0],5))
    #ax.set_xticks(np.arange(0.5,dist[0]+0.5,5))
    #ax.set_xticklabels(np.arange(1,dist[0],5))
    #ax.set_xticks(np.arange(0.5,dist[3]+0.5,30))
    #ax.set_xticklabels(np.arange(1,dist[3],30))
    #ax.set_yticks(np.arange(0.5,dist[3],30))
    #ax.set_yticklabels(np.arange(1,dist[3],30))
    #if idx < 5:
    #    ax.set_ylabel('Monomer (index)')
    #if idx % 5 == 4:
    #    ax.set_xlabel('Hns pole (index)')
    #ax.set_title(fr'$n_{{hns}}={dist[0]}, \phi_c={dist[2]}, a_c={dist[1]}$')
#fig.suptitle(
#    r"Normalized number of contacts between monomers and hns cores in $10^5$ #counts"
#)
#fig.tight_layout(h_pad=1, w_pad=0)#rect=[0, 0, .9, 1])
fig.savefig("PersistenceLength.pdf")

### Test looping

In [ ]:
dir_contacts = np.array(
    [[0, 1, 0, 0, 0],
     [0, 0, 1, 1, 0],
     [1, 0, 0, 0, 0],
     [0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0],
     [0, 0, 1, 0, 1],
     [1, 0, 0, 1, 0]]
    )
dir_contacts

In [ ]:
np.matmul(dir_contacts, dir_contacts.T)

In [ ]:
dir_contacts_up = np.triu(dir_contacts)  # upper triangle
dir_contacts_diag = np.identity(n_atoms, dtype=int)
# dropping diagonal elements:
dir_contacts_up = dir_contacts_up - dir_contacts_diag
# ensuring all diagonal elements are either 0 or 1:
dir_contacts_up[dir_contacts_up < 0] = 0
dir_contacts_low = dir_contacts_up.T  # lower triangle: symmetry!
dir_contacts = dir_contacts_up + dir_contacts_low + dir_contacts_diag
# Creating the full direct contacts matrix
dir_contacts = contact_matrix(dir_contacts)
if save_to is not None:
    output = save_to[0] + \
        f"random_direct_contact-natoms{n_atoms}" + save_to[1]
    np.save(output, dir_contacts)
return dir_contacts

### slicing over different

In [ ]:

t_choice = [  # upper 1.03*10**8
    [3*10**6, 2.53*10**8, r'$\Delta t/\tau=25\times 10^7$'],
    [0.03*10**8, 1.03*10**8, r'$\Delta t/\tau=10\times 10^7$'],
    [0.53*10**8, 1.03*10**8, r'$\Delta t/\tau=5\times 10^7$'],
    [0.83*10**8, 1.03*10**8, r'$\Delta t/\tau=2\times 10^7$'],
    [0.93*10**8, 1.03*10**8, r'$\Delta t/\tau=1\times 10^7$'],
    [0.98*10**8, 1.03*10**8, r'$\Delta t/\tau=0.5\times 10^7$'],
    [1.02*10**8, 1.03*10**8, r'$\Delta t/\tau=0.1\times 10^7$']
]
t_choice = [  # upper 2.03*10**8
    [3*10**6, 2.53*10**8, r'$\Delta t/\tau=25\times 10^7$'],
    [1.03*10**8, 2.03*10**8, r'$\Delta t/\tau=10\times 10^7$'],
    [1.53*10**8, 2.03*10**8, r'$\Delta t/\tau=5\times 10^7$'],
    [1.83*10**8, 2.03*10**8, r'$\Delta t/\tau=2\times 10^7$'],
    [1.93*10**8, 2.03*10**8, r'$\Delta t/\tau=1\times 10^7$'],
    [1.98*10**8, 2.03*10**8, r'$\Delta t/\tau=0.5\times 10^7$'],
    [2.02*10**8, 2.03*10**8, r'$\Delta t/\tau=0.1\times 10^7$']
]
t_choice = [  # upper 1.53*10**8
    [3*10**6, 2.53*10**8, r'$\Delta t/\tau=25\times 10^7$'],
    [0.53*10**8, 1.53*10**8, r'$\Delta t/\tau=10\times 10^7$'],
    [1.03*10**8, 1.53*10**8, r'$\Delta t/\tau=5\times 10^7$'],
    [1.33*10**8, 1.53*10**8, r'$\Delta t/\tau=2\times 10^7$'],
    [1.43*10**8, 1.53*10**8, r'$\Delta t/\tau=1\times 10^7$'],
    [1.48*10**8, 1.53*10**8, r'$\Delta t/\tau=0.5\times 10^7$'],
    [1.52*10**8, 1.53*10**8, r'$\Delta t/\tau=0.1\times 10^7$']
]

In [ ]:
t_choice = [  # upper 1.53*10**8
    [0, 10*10**5, r'$\Delta t/\tau=25\times 10^7$'],
    [5*10**5, 10*10**5, r'$\Delta t/\tau=10\times 10^7$'],
    [1*10**5, 5*10**5, r'$\Delta t/\tau=5\times 10^7$'],
    [5*10**4, 10*10**4, r'$\Delta t/\tau=2\times 10^7$'],
    [2.5*10**4, 5*10**4, r'$\Delta t/\tau=1\times 10^7$'],
    [1.25*10**4, 2.5*10**4, r'$\Delta t/\tau=0.5\times 10^7$'],
    [0, 10**5, r'$\Delta t/\tau=0.1\times 10^7$']
]

In [ ]:
fig, axes = plt.subplots(7,1, figsize=(16,36))
fig.suptitle(r"$t_{final}/\tau=15.3\times 10^7$",y=0.98)
color_palette = 'flare'
#color_palette = 'tab10'
#color_palette = ['orange', 'darkgoldenrod', 'forestgreen']
save_to = './'
ext = 'pdf'
plot_context = 'talk'
font_scale = 2
height = 4
aspect = 2*1.618
ylabel_pad = 50
rc_params= {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'axes.grid.axis': 'both',
    'axes.grid.which': 'both',
    'xtick.top': True,
    'xtick.direction': 'in',
    'ytick.right': True,
    'ytick.direction': 'in',
}
font_family = 'sans-serif'
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)
for ax, (t_i, t_f, dt) in zip(axes,t_choice):
    gyr_sliced = gyr.loc[(gyr['t']>=t_i)&(gyr['t']<=t_f),:]
    gyr_mean = np.round(gyr_sliced['gyr'].mean(),2)
    ax = sns.lineplot(
        data=gyr_sliced,
        x='t',
        y='gyr',
        ci=None,
        palette=color_palette,
        ax=ax,
        label=dt,
    )
    ax.axhline(
        gyr_mean, color='royalblue',
        label=rf'$\langle \sqrt{{R^2_g}}\rangle={gyr_mean}$',
        lw=2, ls='-'
    )
    ax.legend(facecolor='white', loc='upper left', ncol=2)
    ax.set_ylabel(r'$\sqrt{R^2_g}/\sigma$')
    ax.set_xlabel(r'$t/\tau$')
fig.tight_layout(w_pad=1, h_pad=0)
plt.show()
fig.savefig("gyrTDna-slices.pdf")

### Semi-log to show reaching equilibrium

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(16,18))
color_palette = 'flare'
save_to = './'
ext = 'pdf'
plot_context = 'talk'
font_scale = 2
height = 4
aspect = 2*1.618
ylabel_pad = 50
rc_params= {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'axes.grid.axis': 'both',
    'axes.grid.which': 'both',
    'xtick.top': True,
    'xtick.direction': 'in',
    'ytick.right': True,
    'ytick.direction': 'in',
}
font_family = 'sans-serif'
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)
axes[0] = sns.lineplot(
    data=gyr,
    x='t',
    y='gyr',
    ci=None,
    palette=color_palette,
    ax=axes[0]
)
fig.suptitle(r"$\Delta t_{sim}/\tau=28.7\times 10^7$",y=0.92)
axes[0].set_xlabel(r'$t/\tau$')
axes[0].axhline(gyr_mean, color='royalblue', label=r'$\langle \sqrt{R^2_g}\rangle$',
                lw=2, ls='-')
axes[0].legend(facecolor='white', loc='upper right')
axes[0].set_ylabel(r'$\sqrt{R^2_g}/\sigma$')
axes[1].semilogx(
    gyr['t'],
    gyr['gyr']
)
axes[1].set_xlabel(r'$\log(t/\tau)$')
axes[1].axhline(gyr_mean, color='royalblue', label=r'$\langle \sqrt{R^2_g}\rangle$',
                lw=2, ls='-')
axes[1].legend(facecolor='white', loc='upper right')
axes[1].set_ylabel(r'$\sqrt{R^2_g}/\sigma$')
plt.show()
fig.savefig("gyrTDna-semiLogX.pdf")